<a href="https://colab.research.google.com/github/SharovarovAleksandr/Academy/blob/main/Lesson66/Lesson66_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [ ]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/Colab Notebooks/Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=10, target_size=(150, 150))
VALIDATION_DIR = "/content/drive/MyDrive/Colab Notebooks/Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, batch_size=10, target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 212 images belonging to 2 classes.


In [ ]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

<ipython-input-7-6272b23e5a0b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
132/132 [==============================] - ETA: 0s - loss: 0.7801 - acc: 0.5635

132/132 [==============================] - 195s 1s/step - loss: 0.7801 - acc: 0.5635 - val_loss: 0.4155 - val_acc: 0.8538
Epoch 2/10
132/132 [==============================] - ETA: 0s - loss: 0.4271 - acc: 0.8357

132/132 [==============================] - 19s 145ms/step - loss: 0.4271 - acc: 0.8357 - val_loss: 0.2664 - val_acc: 0.9575
Epoch 3/10
132/132 [==============================] - ETA: 0s - loss: 0.3048 - acc: 0.8943

132/132 [==============================] - 19s 147ms/step - loss: 0.3048 - acc: 0.8943 - val_loss: 0.1730 - val_acc: 0.9528
Epoch 4/10
132/132 [==============================] - ETA: 0s - loss: 0.3148 - acc: 0.8829

132/132 [==============================] - 19s 144ms/step - loss: 0.3148 - acc: 0.8829 - val_loss: 0.1730 - val_acc: 0.9481
Epoch 5/10
132/132 [==============================] - ETA: 0s - loss: 0.2652 - acc: 0.9027

132/132 [==============================] - 19s 142ms/step - loss: 0.2652 - acc: 0.9027 - val_loss: 0.0883 - val_acc: 0.9764
Epoch 6/10
132/132 [==============================] - ETA: 0s - loss: 0.1985 - acc: 0.9331

132/132 [==============================] - 20s 151ms/step - loss: 0.1985 - acc: 0.9331 - val_loss: 0.0572 - val_acc: 0.9811
Epoch 7/10
132/132 [==============================] - ETA: 0s - loss: 0.2054 - acc: 0.9240

132/132 [==============================] - 19s 144ms/step - loss: 0.2054 - acc: 0.9240 - val_loss: 0.0559 - val_acc: 0.9764
Epoch 8/10
132/132 [==============================] - 18s 137ms/step - loss: 0.2361 - acc: 0.9110 - val_loss: 0.0677 - val_acc: 0.9858
Epoch 9/10
132/132 [==============================] - 19s 142ms/step - loss: 0.1992 - acc: 0.9232 - val_loss: 0.1005 - val_acc: 0.9481
Epoch 10/10
132/132 [==============================] - ETA: 0s - loss: 0.1930 - acc: 0.9224

132/132 [==============================] - 19s 142ms/step - loss: 0.1930 - acc: 0.9224 - val_loss: 0.0397 - val_acc: 0.9906


In [12]:
import cv2 as cv2
import numpy as np
from keras.models import load_model
from google.colab.patches import cv2_imshow

model=load_model("/content/model2-002.model")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/Dataset/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(im)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2_imshow(im)
    key = cv2.waitKey(10)
    
    if key == 'q': 
        break
cap.release()
cv2.destroyAllWindows()